# 1. nn.GRU 基本介绍

`nn.GRU `是 `门控循环单元`（Gated Recurrent Unit） 的实现。
它是 `LSTM `的简化版，只有 `更新门`（update gate） 和 `重置门`（reset gate），没有 LSTM 的 `细胞状态 c`。
- 相比 LSTM：

    - 参数更少，计算更快
    
    - 效果接近甚至优于 LSTM（具体要看任务）

### 定义：

In [ ]:
torch.nn.GRU(
    input_size,       # 输入特征维度
    hidden_size,      # 隐藏层维度
    num_layers=1,     # 堆叠多少层 GRU
    bias=True,        # 是否使用偏置
    batch_first=False,# True: 输入 (batch, seq, feature)
    dropout=0.0,      # 层与层之间的 dropout
    bidirectional=False # 是否为双向 GRU
)


# 2. 输入 & 输出格式

- 假设：

    - batch_size = B
    - seq_len = T
    - input_size = D_in
    - hidden_size = D_h

- 输入

    - input：
        - (T, B, D_in) 若 batch_first=False
        - (B, T, D_in) 若 batch_first=True
        - h_0（可选，初始隐藏状态）：(num_layers * num_directions, B, D_h)

- 输出

    - output：
        - (T, B, num_directions * D_h) 或 (B, T, num_directions * D_h).包含每个时间步的隐藏状态。
        
        - h_n：(num_layers * num_directions, B, D_h)
最后一个时间步的隐藏状态（没有 LSTM 的 c_n）。

---
# 3. 代码示例
### (1) 单层 GRU

In [ ]:
import torch
import torch.nn as nn

# 参数
input_size = 5     # 每个时间步的输入特征维度
hidden_size = 3    # 隐藏层维度
seq_len = 4
batch_size = 2

# 定义 GRU
gru = nn.GRU(input_size, hidden_size, batch_first=True)

# 输入 (batch, seq, feature)
x = torch.randn(batch_size, seq_len, input_size)

# 前向传播
output, h_n = gru(x)

print("输入 x:", x.shape)          # [2, 4, 5]
print("输出 output:", output.shape) # [2, 4, 3]
print("最终隐藏状态 h_n:", h_n.shape) # [1, 2, 3]


### (2) 多层 GRU

In [ ]:
gru = nn.GRU(input_size=5, hidden_size=3, num_layers=2, batch_first=True)
x = torch.randn(2, 4, 5)
output, h_n = gru(x)

print("output:", output.shape)   # [2, 4, 3]
print("h_n:", h_n.shape)         # [2, 2, 3] (2 层, batch=2, hidden=3)


### (3) 双向 GRU

In [ ]:
gru = nn.GRU(input_size=5, hidden_size=3, num_layers=1, bidirectional=True, batch_first=True)
x = torch.randn(2, 4, 5)
output, h_n = gru(x)

print("output:", output.shape)   # [2, 4, 6] (hidden_size * 2)
print("h_n:", h_n.shape)         # [2, 2, 3] (2 directions, batch=2, hidden=3)


# 4. GRU 的优势

- 比 LSTM 更轻量（参数更少，速度更快）

- 没有细胞状态 c，实现更简洁

- 在很多任务上表现与 LSTM 差不多甚至更好

- 但在一些需要长期记忆的任务中，LSTM 可能更优